In [1]:
import os
os.chdir("../")

## SCALABILITY, MAINTAINABILITY AND ROBUSTNESS ARE THE IMPORTANT ASPECTS OF A PROJECT 

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cifar100classifier.constants import *
from cifar100classifier.utils import read_yaml, create_directories,get_current_time_stamp

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return callback_config

In [5]:
import os
import tensorflow as tf

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        
        """TensorBoard is a tool for providing the measurements and visualizations 
        needed during the machine learning workflow.
        
        This callback logs events for TensorBoard, including:
            Metrics summary plots,
            Training graph visualization,
            Weight histograms,
            Sampled profiling"""
                
        timestamp = get_current_time_stamp()  #time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir) # tensorboard callbacks

    @property
    def _create_ckpt_callbacks(self):
        
        """Callback to save the Keras model or model weights at some frequency."""
        
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    @property
    def _create_early_stopping_callbacks(self):
        
        """Stop training when a monitored metric has stopped improving."""
        
        return tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.1,
            patience=5,
            verbose=1,
            mode='min',
            baseline=None,
            restore_best_weights=True
        )
        
    @property
    def _create_reduce_LR_onplateau_callbacks(self):
        
        """Reduce learning rate when a metric has stopped improving."""
        
        return tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            verbose=1,
            mode='min',
            min_delta=0.1,
            cooldown=0,
            min_lr=1e-6
        )

    def get_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_early_stopping_callbacks,
            self._create_reduce_LR_onplateau_callbacks
        ]


In [6]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=callbacks_config)
    callback_list = prepare_callbacks.get_callbacks()
    
except Exception as e:
    raise e

[2023-03-05 04:56:31,836: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2023-03-05 04:56:32,089: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-05 04:56:32,592: INFO: common]: created directory at: artifacts
[2023-03-05 04:56:32,716: INFO: common]: created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2023-03-05 04:56:32,717: INFO: common]: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
